In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import mira

In [ ]:
# https://github.com/cistrome/MIRA
data = sc.read_h5ad('mira-datasets/e18_10X_brain_dataset/e18_mouse_brain_10x_dataset.ad')

In [ ]:
rna_data = data[:, data.var.feature_types == 'Gene Expression']
atac_data = data[:, data.var.feature_types == 'Peaks']

In [ ]:
# Basic preprocessing steps
rna_data.var.index = rna_data.var.index.str.upper()
rna_data.var_names_make_unique()
rna_data = rna_data[:, ~rna_data.var.index.str.startswith('GM')]

sc.pp.filter_cells(rna_data, min_counts = 400)
sc.pp.filter_genes(rna_data, min_cells=15)

rna_data.var['mt'] = rna_data.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(rna_data, qc_vars=['mt'], percent_top=None,
                           log1p=False, inplace=True)

rna_data = rna_data[rna_data.obs.pct_counts_mt < 15, :]
rna_data = rna_data[rna_data.obs.n_genes_by_counts < 8000, :]
sc.pp.filter_genes(rna_data, min_cells=15)

rna_data.raw = rna_data # save raw counts
sc.pp.normalize_total(rna_data, target_sum=1e4)
sc.pp.log1p(rna_data)

sc.pp.highly_variable_genes(rna_data, min_disp = -0.1)
rna_data.layers['norm'] = rna_data.X # save normalized count data
rna_data.X = rna_data.raw.X # and reload raw counts
rna_data = rna_data[:, rna_data.var.highly_variable] 
rna_data.var['exog_feature'] = rna_data.var.highly_variable # set column "exog_features" to all genes that met dispersion threshold
rna_data.var.highly_variable = (rna_data.var.dispersions_norm > 0.8) & rna_data.var.exog_feature # set column "highly_variable" to genes that met first criteria and dispersion > 0.8

overlapping_barcodes = np.intersect1d(rna_data.obs_names, atac_data.obs_names) # make sure barcodes are matched between modes
atac_data = atac_data[[i for i in overlapping_barcodes],:]

In [ ]:
rna_model = mira.topic_model.ExpressionTopicModel.load('mira-datasets/e18_10X_brain_dataset/e18_mouse_brain_10x_rna_model.pth')
atac_model = mira.topic_model.AccessibilityTopicModel.load('mira-datasets/e18_10X_brain_dataset/e18_mouse_brain_10x_atac_model.pth')

In [ ]:
rna_model.predict(rna_data)
atac_model.predict(atac_data, batch_size=128)

In [ ]:
atac_model.get_umap_features(atac_data, box_cox = 0.5)
rna_model.get_umap_features(rna_data, box_cox = 0.5)
rna_data, atac_data = mira.utils.make_joint_representation(rna_data, atac_data)
rna_model.impute(rna_data)

In [ ]:
main_barcodes = pd.read_csv("mira-datasets/e18_10X_brain_dataset/e18_mouse_brain_10x_main_barcodes.csv", index_col=0, header=0, names=["barcodes"])

In [ ]:
rna_main = rna_data[list(main_barcodes["barcodes"])]
atac_main = atac_data[list(main_barcodes["barcodes"])]

exp_topic_ordered = [f'topic_{i}' for i in     # included topics, custom order
                     [1, 3, 19, 20, 10, 13, 9, 0, 15, 8, 16, 18, 7, 5, 2, 21, 4, 6, 14, 12]]
acc_topic_ordered = [f'topic_{i}' for i in     # included topics, custom order
                     [11, 5, 6, 2, 8, 7, 0, 9, 10, 1, 12, 4, 3]]

In [ ]:
df = data[:, data.var.feature_types == 'Peaks'].to_df()

In [ ]:
# original = df.head(100)
# new = atac_main.obs.head(100)

# to_save = original.join(new)
# to_save = to_save.filter(regex='topic')
# to_save *= 1000
# to_save.reset_index().to_json('output/obs.json', orient='records')

to_save = atac_main.obs.head(500)
to_save *= 1000
to_save.reset_index().to_json('output/obs.json', orient='records')

# atac_main.varm['topic_feature_compositions']
# atac_main.obsm#['X_joint_umap_features']


In [ ]:
# sc.pp.neighbors(rna_main, use_rep='X_joint_umap_features') #, metric='manhattan')

In [ ]:
# sc.tl.umap(rna_main, min_dist = 0.3, negative_sample_rate=5)

In [ ]:
# atac_main.obsm['X_umap'] = rna_main.obsm['X_umap']